# Boltzmanns fördelningslag

Detta är Python-programmet som tillhör inlämningsuppgiften Boltzmanns fördelningslag på KFKA05

Det är en så kallad Jupyter notebook som består av två celler med kod. Du måste exekvera båda cellerna (i ordning) för att programmet ska starta, och sen scrolla ner längst ner på sidan till den blåa rutan med bl.a. diagram.

Den första kodcellen innehåller själva funktionsdefinitionerna som kan vara intressanta att titta på (men inget synbart händer när de körs). Den andra cellen innehåller framför allt koden till det grafiska gränssnittet och är bara intressant om du vill koda liknande saker själv.

In [ ]:
#Imports and function definitions

%pip install ipywidgets
import math
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy.optimize import fsolve
R=8.3145
NA=6.022e23
kB=R/NA

#Functions used

def safelog(x): return np.ma.log(x).filled(-1e9)  # The value of log(0) doesn't matter as it's multiplied by 0 anyway

def pop(levels, T):
    T+=1e-3  #Avoid numerical errors when T=0 by adding a negligible value
    q=np.sum(np.exp(-levels/T))
    p=np.exp(-levels/T)/q
    U=np.sum(p*levels)
    S=-np.sum(p*safelog(p))
    return (q,p,U,S)


def setEnergy(U,levels):  #Returns the temperature that corresponds to energy U
    return fsolve(lambda T: pop(levels,T)[2]-float(U), 1.0)

def energytransfer(deltaU, TA, TB, levelsA, levelsB):   #Move deltaU (which can be positive or negative) from system A to system B
    (_,_,UA,_)=pop(levelsA,TA)
    (_,_,UB,_)=pop(levelsB,TB)
    UA-=deltaU
    UB+=deltaU
    if UA>=0 and UB>=0:
        TA=setEnergy(UA,levelsA)
        TB=setEnergy(UB,levelsB)
        return (True,TA,TB)
    else:
        return (False,TA,TB)

def equidistant(distance):
    return np.arange(1000)*float(distance)

def sublevels(numsublevels, maindistance, subdistance):
    lev=[]
    for i in range(1000//numsublevels):
        for j in range(numsublevels):
            lev.append(i*float(maindistance)+j*float(subdistance))
    lev.sort()
    return np.array(lev)

In [2]:
#Start the app with these default settings

levels=[equidistant(50),equidistant(50)]    #List of two systems, unit K
Tinit=50
nshow=11     #number of levels shown
continuous=True   #Update graphs while sliding temperatures. Set to false if lagging is too severe.

def popformat(x):
    if x>0.1:
        return "%.3g"%x
    elif x>0.01:
        return "%.3f"%x
    elif x>1e-100:
        return "%.1e"%x
    else:
        return "0"

def update(sysnr, T, relative):
    (q,p,U,S)=pop(levels[sysnr],T)
    if relative:
        p/=p[0]
    restext='S/k (particle): %.4f\nS (molar): %.4f J/(K mol)\nq: %.4f\nU/k (particle): %.4f K\nU (molar): %.4f kJ/mol\n'%(S,S*R,q,U,U*R*1e-3)
    plt.xlim([0,1])
    bars=plt.barh(levels[sysnr][:nshow],p[:nshow],height=0.5*(levels[sysnr][1]-levels[sysnr][0]))
    plt.bar_label(bars,fmt=popformat)
    plt.annotate(restext, xy=(0.6, 0.7), xycoords='axes fraction')
    plt.xlabel('Population')
    plt.ylabel(r'Energy $E_i\; /\; k_B$ / (K$^{-1}$)')
    plt.show()
    

def update1(T,relative):
    update(0,T,relative)

def update2(T,relative):
    update(1,T,relative)

def transfer(b):
    ene = -edit.value*1e3/R if b == A.transferbutton else edit.value*1e3/R
    (success,TA,TB)=energytransfer(ene, A.readout.value, B.readout.value,levels[0], levels[0])
    if success:
        A.readout.value=TA
        B.readout.value=TB
        statuslabel.value=""
    else:
        statuslabel.value="IMPOSSIBLE!"

def changelevels(syst):
    global levels
    nsub=syst.levlist.value
    if nsub==1:
        v=syst.main.value
        newlevels=equidistant(syst.main.value)
    else:
        newlevels=sublevels(nsub,syst.main.value,syst.sub.value)
    if syst==A:
        levels[0]=newlevels
    else:
        levels[1]=newlevels
    syst.readout.value+=0.00001 #to force update even if T doesn't change
    

def levlistclick(b):
    syst = A if b['owner']==A.levlist else B
    syst.sub.disabled=(syst.levlist.value==1)
    changelevels(syst)

def levtextclick(b):
    syst = A if (b['owner']==A.main or b['owner']==A.sub) else B
    changelevels(syst)


#Define each set of widgets Structure
class Systemwidgets:
    def __init__(self,isleft):
       self.header=widgets.HTML(value="<h2><center>System "+("A" if isleft else "B")+"</center></h2>")
       self.levlist=widgets.Dropdown(options=[('Equidistant',1),('2 sublevels',2),('3 sublevels',3),('4 sublevels',4),('5 sublevels',5)],layout = widgets.Layout(width='100px'))
       self.main=widgets.BoundedIntText(description="",value=50,min=1,max=200,layout = widgets.Layout(width='50px'))
       self.sub=widgets.BoundedIntText(description="",value=10,min=1,max=200,layout = widgets.Layout(width='50px'))
       self.sub.disabled = True
       self.slider = widgets.FloatSlider(description='Temp (K)',value=Tinit,min=0,max=1000,step=1, continuous_update = continuous,readout=False)
       self.readout = widgets.BoundedFloatText(value=self.slider.value,min=self.slider.min,max=self.slider.max,step=0.1,layout=widgets.Layout(width='70px'))
       self.box = widgets.Checkbox(False, description='relative')
       self.transferbutton=widgets.Button(description='<<' if isleft else '>>',layout=widgets.Layout(margin=("0px 0px 0px auto" if isleft else ""), width="20%"),button_style='info')
       self.transferbutton.on_click(transfer)
       self.levlist.observe(levlistclick)
       self.main.observe(levtextclick)
       self.sub.observe(levtextclick)

       def slider_changed(change): #only change readout if the value really changed, that is sliding, not just updating the slider when T changed
           if abs(float(change['new'])-round(self.readout.value))>0.5:
               self.readout.value = float(change['new'])

       def box_changed(change):
           self.readout.value = round(self.readout.value, 2)
           self.slider.value = round(self.readout.value)

       self.slider.observe(slider_changed, names='value')
       self.readout.observe(box_changed, names='value')
        
       if isleft:
          self.output=widgets.interactive_output(update1, {'T':self.readout, 'relative' : self.box})
       else:
          self.output=widgets.interactive_output(update2, {'T':self.readout, 'relative' : self.box})
       self.defbox=widgets.HBox([self.levlist,widgets.Label("Spacing: Main:"),self.main,widgets.Label("K"),
                                 widgets.HTML(value="&nbsp;&nbsp;&nbsp;&nbsp;"),widgets.Label("Sub:"),self.sub,widgets.Label("K")])

A=Systemwidgets(True)
B=Systemwidgets(False)
statuslabel=widgets.Label(value="")
edit=widgets.BoundedFloatText(value=0.1,min=0,max=10.0,step=0.01,description='',disabled=False, layout = widgets.Layout(width='70px'))

widgets.GridBox(children=[A.header,widgets.Label(),B.header,
                          A.defbox,widgets.Label(),B.defbox,
                          widgets.HBox([A.slider,A.readout]),widgets.Label('Heat flow: '),widgets.HBox([B.slider,B.readout]),
                          widgets.HBox([A.box,A.transferbutton]),widgets.HBox([edit, widgets.Label(value="kJ/mol")]),widgets.HBox([B.transferbutton,B.box]),
                          A.output, statuslabel, B.output],
               layout=widgets.Layout(
                            width='100%',
                            grid_template_columns='1fr auto 1fr',
                            grid_template_rows='auto 40px auto 40px auto auto',
                            grid_gap='5px 2px',
                            border="2px solid blue")
       )


GridBox(children=(HTML(value='<h2><center>System A</center></h2>'), Label(value=''), HTML(value='<h2><center>S…